<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/text-similarity-works/13_code_keyword_sentence_extraction_and_highlight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [ ]:
!pip -q install spacy
!python -m spacy download en_core_web_sm

!pip install pillow

!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
!pip install -U pdftotext
!pip install PyPDF2
!pip install fitz
!pip install PyMuPDF

In [3]:
import pandas as pd
import re
import os
import glob
import difflib
import pickle

import fitz
import pdftotext
from PyPDF2 import PdfFileReader, PdfFileWriter

import spacy
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English

from keyword_extractor import *
from concurrent import futures

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))

In [4]:
!mkdir -p pdf-files
!mkdir -p txt-files
!mkdir -p ocr-pdf-files

In [5]:
# define directory path after creating it
pdf_files_path = "pdf-files"
txt_files_path = "txt-files"
ocr_pdf_files_path = "ocr-pdf-files"

MAX_WORKERS = 20

##Core Functions

In [57]:
def split_pdf(pdf_path):
  pdf_in_file = open(pdf_path, "rb")
  pdf = PdfFileReader(pdf_in_file)
  pdf_list = []
  for page in range(pdf.numPages):
      inputpdf = PdfFileReader(pdf_in_file)
      output = PdfFileWriter()
      output.addPage(inputpdf.getPage(page))
      with open(f"{pdf_files_path}/page-{page}.pdf", "wb") as outputStream:
          output.write(outputStream)
          pdf_list.append(f"page-{page}.pdf")
  return pdf_list


def extract_text_from_pdf(pdf_list):
    txt_file_list = []
    i = 0
    for pdf_file in pdf_list:
        with open(os.path.join(pdf_files_path, pdf_file), "rb") as f:
            pdf = pdftotext.PDF(f)

        # Read all the text into one string
        pdf_text = "\n\n".join(pdf)

        # write text into file
        with open(f"{txt_files_path}/page-{str(i)}.txt", "a") as f:
            f.write(pdf_text)
        txt_file_list.append(f"{txt_files_path}/page-{str(i)}.txt")
        i += 1
    return txt_file_list


def get_opt_pattern(icd_10_code):
  # create alternate pattern
  code_arr = icd_10_code.split(".")
  if len(code_arr) > 1:
    code1 = f"{code_arr[0]}. {code_arr[1]}"
    code2 = f"{code_arr[0]} .{code_arr[1]}"
    code3 = f"{code_arr[0]} . {code_arr[1]}"
    return [code1, code2, code3]
  else:
    return icd_10_code


def isExactMatch(page, term, clip, fullMatch=False, caseSensitive=False):
  # clip is an item from page.search_for(term, quads=True)
  termLen = len(term)
  termBboxLen = max(clip.height, clip.width)
  termfontSize = termBboxLen/termLen
  f = termfontSize*2

  #clip = clip.rect

  validate = page.get_text("blocks", clip = clip + (-f, -f, f, f), flags=0)[0][4]
  flag = 0
  if not caseSensitive:
      flag = re.IGNORECASE

  matches = len(re.findall(f'{term}', validate, flags=flag)) > 0
  if fullMatch:
      matches = len(re.findall(f'\\b{term}\\b', validate))>0
  return matches

def highlight_icd_code_and_keyword(icd10_code_dict, icd9_code_dict, icd_keyword_list=None, pdf_file_name=None, cords_file_name=None):
  pdf_file = fitz.open(pdf_file_name)

  def highlight_pdf(highlight, icd10_code, code_type):
    cords_list = []
    for inst in highlight:
      highlight = page.add_highlight_annot(inst)
      if code_type == "ICD-9":
        highlight.set_colors(stroke=[1, 0.5, 0.8]) # light red color (r, g, b)
      highlight.update()
      highlight = page.search_for(icd10_code)
      cords_list.append(highlight)

    if cords_list:
      code_cors_output = f"Page-{page_num}: {icd10_code} : {cords_list}"
      txt_output_file_name.write("%s\n" % code_cors_output)

  # create file to write cordinate 
  txt_output_file_name = open(cords_file_name, "a")

  for page_num, page in enumerate(pdf_file):

    # highlight ICD-10 code
    if page_num in icd10_code_dict:
      for code in icd10_code_dict[page_num]:
        highlight = page.search_for(code)
        if len(highlight) == 0:
          alternate_code_list = get_opt_pattern(code)
          for alt_code in alternate_code_list:
            highlight = page.search_for(alt_code)
            # highlight pdf for option pattern
            highlight_pdf(highlight, alt_code, code_type="ICD-10")
        # highlight pdf for main pattern   
        highlight_pdf(highlight, code, code_type="ICD-10")

    # highlight ICD-9 code
    if page_num in icd9_code_dict:
      for code in icd9_code_dict[page_num]:
        highlight = page.search_for(code)
        if len(highlight) == 0:
          alternate_code_list = get_opt_pattern(code)
          for alt_code in alternate_code_list:
            highlight = page.search_for(alt_code)
            # highlight pdf for option pattern
            highlight_pdf(highlight, alt_code, code_type="ICD-9")
        # highlight pdf for main pattern   
        highlight_pdf(highlight, code, code_type="ICD-9")

  # highlight ICD key phrase
  if icd_keyword_list:
    cords_list = []
    for key_phrase in icd_keyword_list:
      # search key phrase for every page
      for page in pdf_file:
        coordinates = page.search_for(key_phrase)
        #print(f"Keyword: {keyword}, Length: {len(coordinates)}")
        
        for inst in coordinates:
          #print(f"Keyword: {keyword}, inst: {inst}")
          #if isExactMatch(page, keyword, inst, fullMatch=True, caseSensitive=False):
          highlight = page.add_highlight_annot(inst)
          highlight.set_colors(stroke=[1, 0.8, 0.8])
          highlight.update()
          highlight = page.search_for(key_phrase)
          cords_list.append(highlight)

    if cords_list:
      keyword_cors_output = f"Page-{page_num}: {key_phrase} : {cords_list}"
      txt_output_file_name.write("%s\n" % keyword_cors_output)
      #print(f"Page-{page_num}: ", highlight, end='\n')

  txt_output_file_name.close()

  pdf_output_file_name = f"{pdf_file_name.split('.')[0]}_output.pdf"
  pdf_file.save(pdf_output_file_name, garbage=4, deflate=True, clean=True)

  return pdf_output_file_name, cords_file_name


def filter_unwanted_code(code_list, page_text):
    filtered_code_list = []
    # if re.search("ICD", page_text):
    # match_list = re.findall("(ICD-[0-9][a-zA-z]*\-.+)[ ]", page_text)
    match_list = re.findall("(IC[(A-z)]-[0-9][a-zA-z]*\-.+)[ ]", page_text)
    # print("Match list:\n", match_list)
    for found_code in match_list:
        for code in code_list:
            if code in found_code:
                filtered_code_list.append(code)
    return filtered_code_list


def search_icd_code(txt_list, nlp, code_type):
    pdf_page_vocab = {}
    for txt_file in txt_list:
        with open(txt_file, "r") as f:
            page_txt = f.read()
            # filter the page that have line number instead of code
            if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
                doc = nlp(page_txt)
                code_list = [ent.text for ent in doc.ents]
                page_number = 0
                if len(code_list) != 0:
                    page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
                    pdf_page_vocab[page_number] = code_list
                    # print(f"Page[{txt_file.split('/')[1]}]: {code_list}")

                # filter the page that dont have ICD string into it
                if code_type == "ICD-9":
                    filtered_code_list = filter_unwanted_code(code_list, page_txt)
                    pdf_page_vocab[page_number] = filtered_code_list
                    # print(f"Page[{txt_file.split('/')[1]}]: {filtered_code_list}")

    return pdf_page_vocab


def get_json_array_list(text_path):
  print(f"Running '{text_path}'")
  json_arr = call(text_path)
  print(f"Got json for '{text_path}'")
  return json_arr


def get_wrong_keyword_dict_with_thread(text_path_list):
  wrong_keyword_dict = {}

  # take care so that unnecessary thread should not be created
  workers = min(MAX_WORKERS, len(text_path_list))
  with futures.ThreadPoolExecutor(max_workers=workers) as executor:
    json_arr_list = executor.map(get_json_array_list, sorted(text_path_list))

  for idx, json_arr in enumerate(json_arr_list):
    wrong_keyword_list = [list(element.values())[0] for element in json_arr]
    if wrong_keyword_list: 
      wrong_keyword_dict[idx] = set(wrong_keyword_list)
  return dict(sorted(wrong_keyword_dict.items(), key=lambda item: item[0]))

def get_wrong_keyword_dict_with_process(text_path_list):
  wrong_keyword_dict = {}

  # take care so that unnecessary thread should not be created
  workers = min(MAX_WORKERS, len(text_path_list))
  with futures.ProcessPoolExecutor(max_workers=4) as executor:
    json_arr_list = executor.map(get_json_array_list, sorted(text_path_list))

  for idx, json_arr in enumerate(json_arr_list):
    wrong_keyword_list = [list(element.values())[0] for element in json_arr]
    if wrong_keyword_list: 
      wrong_keyword_dict[idx] = set(wrong_keyword_list)
  return dict(sorted(wrong_keyword_dict.items(), key=lambda item: item[0]))


def get_wrong_keyword_dict(text_path_list):
  wrong_keyword_dict = {}
  for idx, file_path in enumerate(text_path_list):
    print(idx, file_path)
    json_arr = get_json_array_list(file_path)
    wrong_keyword_list = [list(element.values())[0] for element in json_arr]
    if wrong_keyword_list:
      wrong_keyword_dict[idx] = set(wrong_keyword_list)
  return wrong_keyword_dict


def purge(file_path):
  for f in glob.glob(file_path):
    os.remove(f)

##Prepare keyword/phrase

In [7]:
# Step-0: Load prerequisite instance
# create nlp instance
nlp_keyword = spacy.load('en_core_web_sm')

# loading and updating patterns for ICD-10 code
nlp_code10 = English()
nlp_code10.add_pipe("entity_ruler").from_disk("./icd10_code_patterns-v3.jsonl")

# loading and updating patterns for ICD-9 code
nlp_code9 = English()
nlp_code9.add_pipe("entity_ruler").from_disk("./icd9_code_patterns-v1.jsonl")

In [8]:
# Step-1: spliting pdf file
pdf_file_name = "Redacted_Sample.pdf"
pdf_list = split_pdf(pdf_file_name)

# Step-2: Extracting text from pdf
txt_list = extract_text_from_pdf(pdf_list)

In [9]:
# Step-3: Searching ICD-10 code
page_code10_dict = search_icd_code(txt_list, nlp_code10, code_type="ICD-10")
page_code10_dict

In [ ]:
# Step-4: Searching ICD-9 code
page_code9_dict = search_icd_code(txt_list, nlp_code9, code_type="ICD-9")
page_code9_dict

In [12]:
icd_code_kerword_df = pd.read_csv("icd_10_code_keywords.csv")
icd_code_kerword_df.head()

,Code,Keyword
0,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A00.9,"Cholera, unspecified"
3,A01.00,"Typhoid fever, unspecified"
4,A01.01,Typhoid meningitis


In [13]:
def insert_dot(code, index):
  if not "." in code:
    if len(code) > 3:
      return code[:index] + "." + code[index:]
    else:
      return code[:index] + "." + code[index:] + "0"
  else:
    return code

print(insert_dot("G0439", 3))

G04.39


In [61]:
def get_keyword_list(code_dict):
  keyword_list = []
  icd_kerword_list = []
  # get code list
  code_list = [value for key, value in page_code10_dict.items()]
  code_list = [[insert_dot(code, 3) for code in temp_list] for temp_list in code_list]
  for t_list in code_list:
    keyword_list.extend(t_list)
  #print(len(keyword_list))
  #print(list(set(keyword_list)))
  # filter dataframe based on code list
  icd_kerword_df = icd_code_kerword_df.loc[icd_code_kerword_df['Code'].isin(list(set(keyword_list)))]
  # create file to write cordinate 
  icd_keyword_found_filename = open("icd_keyword_found.txt", "a")
  for _, row in icd_kerword_df.iterrows():
    icd_kerword_list.append(row["Keyword"])
    #icd_kerword_list.append(row["Keyword"].lower())
    keyword_found_output = f"{row['Code']}: {row['Keyword']}"
    icd_keyword_found_filename.write("%s\n" % keyword_found_output)
  icd_keyword_found_filename.close()
  return icd_kerword_list

In [62]:
icd_kerword_list = get_keyword_list(page_code10_dict)

In [64]:
len(icd_kerword_list)

15

In [63]:
icd_kerword_list

['Benign lipomatous neoplasm of skin and subcutaneous tissue of trunk',
 'Decreased white blood cell count, unspecified',
 'Lipomatosis, not elsewhere classified',
 'Reaction to severe stress, unspecified',
 'Other acute necrotizing hemorrhagic encephalopathy',
 'Migraine with aura, not intractable, without status migrainosus',
 'Acute bronchitis due to Mycoplasma pneumoniae',
 'Unspecified contact dermatitis, unspecified cause',
 'Cervicalgia',
 'Lumbago with sciatica, unspecified side',
 'Congenital ptosis',
 'Palpitations',
 'Lower abdominal pain, unspecified',
 'Encounter for general adult medical examination without abnormal findings',
 'Encounter for examination for admission to educational institution']

##Extract Sentence

In [18]:
# Load your PDF
with open("Redacted_Sample.pdf", "rb") as f:
  pdf = pdftotext.PDF(f)

pdf_text = "\n\n".join(pdf)
# write text into file
with open("ocr-extracted.txt", "a") as f:
  f.write(pdf_text)

print("File is written")

File is written


In [19]:
def extract_sentence(key_phrase_list, sample_text):
  match_lists = []

  for key_phrase in key_phrase_list:
    for word in key_phrase.split():
      if not word.lower() in stop_words:
        match_list = re.findall(f"([^.\n]*?{word}[^.]*\.)", sample_text)
        match_lists.extend(match_list)
  return list(set(match_lists))

In [65]:
def extract_sentence(key_phrase_list, sample_text):
  match_lists = []

  for key_phrase in key_phrase_list:
    match_list = re.findall(f"([^\n]*?(?i){key_phrase}[^.]*\.)", sample_text)
    match_lists.extend(match_list)
  return list(set(match_lists))

In [ ]:
%%time

match_founds = extract_sentence(icd_kerword_list, pdf_text)
len(match_founds)

In [ ]:
cleal_found_list = []
for found in match_founds:
  cleal_found_list.append(found.replace("\n", ""))
cleal_found_list

In [68]:
len(cleal_found_list)

47

In [69]:
# Step-6: Highlighting ICD-10 code and keyword into pdf
pdf_output_file, txt_output_file = highlight_icd_code_and_keyword(page_code10_dict, 
                                                                  page_code9_dict, 
                                                                  cleal_found_list,
                                                                  "Redacted_Sample.pdf", 
                                                                  "Redacted_Sample_cords.txt")
print(f"File[{pdf_output_file}] is saved after highlighting ICD-10 code and keyword")
print(f"Highlighted coordinates are saved into [{txt_output_file}] file.")

File[Redacted_Sample_output.pdf] is saved after highlighting ICD-10 code and keyword
Highlighted coordinates are saved into [Redacted_Sample_cords.txt] file.


##Searching & Highlighting

In [ ]:
# Step-0: Load prerequisite instance
# create nlp instance
nlp_keyword = spacy.load('en_core_web_sm')

# loading and updating patterns for ICD-10 code
nlp_code10 = English()
nlp_code10.add_pipe("entity_ruler").from_disk("./icd10_code_patterns-v3.jsonl")

# loading and updating patterns for ICD-9 code
nlp_code9 = English()
nlp_code9.add_pipe("entity_ruler").from_disk("./icd9_code_patterns-v1.jsonl")

In [ ]:
%%time

for pdf_file in os.listdir(ocr_pdf_files_path):
  pdf_file_name = f"{ocr_pdf_files_path}/{pdf_file}"
  cords_file_name = f"{pdf_file_name.split('.')[0]}_cords.txt"

  # Step-1: splitting pdf file
  pdf_list = split_pdf(pdf_file_name)

  # Step-2: Extracting text from pdf
  txt_list = extract_text_from_pdf(pdf_list)

  # Step-3: Searching ICD-10 code
  icd10_code_dict = search_icd_code(txt_list, nlp_code10, code_type="ICD-10")

  # Step-4: Searching ICD-9 code
  page_code9_dict = search_icd_code(txt_list, nlp_code9, code_type="ICD-9")

  # Step-5: Searching ICD-10 keyword
  wrong_keyword_dict = get_wrong_keyword_dict(txt_list)
  # wrong_keyword_dict = get_wrong_keyword_dict_with_thread(txt_list)
  # wrong_keyword_dict = get_wrong_keyword_dict_with_process(txt_list)
  # print("After sorting:\n", wrong_keyword_dict)

  # Step-6: Highlighting ICD-10 code and keyword into pdf
  pdf_output_file, txt_output_file = highlight_icd_code_and_keyword(icd10_code_dict, 
                                                                    page_code9_dict, 
                                                                    wrong_keyword_dict,
                                                                    pdf_file_name, 
                                                                    cords_file_name)
  print(f"File[{pdf_output_file}] is saved after highlighting ICD-10 code and keyword")
  print(f"Highlighted coordinates are saved into [{txt_output_file}] file.")

  # remove all pdf and text files
  purge("pdf-files/*.pdf")
  purge("txt-files/*.txt")
  pdf_list = []
  txt_list = []

In [ ]:
!rm -rf ocr-pdf-files
!mkdir -p ocr-pdf-files

In [ ]:
purge("ocr-pdf-files/*.txt")
purge("ocr-pdf-files/*_output.pdf")
purge("pdf-files/*.pdf")
purge("txt-files/*.txt")

In [ ]:
!zip output.zip ocr-pdf-files/*_output_cords.txt ocr-pdf-files/*_output_output.pdf